In [1]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *
import numpy as np
import SimpleITK as sitk
import csv
import shutil

In [ ]:
#Script to write csv to use in Deep Learning 

In [3]:
json_path = '/media/deeplearning/Elements/RELEVANCE_Valid_DICOM/relevance_data.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

cpt = 0
for serie in dataset : 
    if 'PET_Screening' in serie : 
        cpt += 1

print(cpt)
print(len(dataset))

206
499


In [4]:
#open csv with informations about dataset
csv_path = '/media/deeplearning/Elements/relevance_data_v2.csv'

with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] 

In [22]:
#PET_Screening only 

data = []
for row in csv_data : 
    if 'PET_Screening' in row : 
        data.append(row)

In [23]:
len(data)

208

In [7]:
#liste of generated nifti by "nifti_builder_from_json" script 

nifti_directory = '/media/deeplearning/Elements/RELEVANCE_NIFTI'
liste_nifti = os.listdir(nifti_directory)

In [8]:
len(liste_nifti)

392

In [24]:
liste = []
type_ = 'PET_Screening'
for row in data : 
    subliste = []
    subliste.append(row[0]) #patient id 
        #add other informations if need 

    study_uid = row[1]
    subliste.append(study_uid)

    subsubliste = []
    for nifti in liste_nifti : 
        if study_uid in nifti: 
            subsubliste.append(nifti)
    if len(subsubliste) == 2 : #or == 3 if there is a mask 
        subsubliste = sorted(subsubliste)
        subliste.append(subsubliste[0])
        subliste.append(subsubliste[1])
            #subliste.append(subsubliste[2]) #add it if there is a mask 
    else : 
        pass
    subliste.append(type_)
    liste.append(subliste)



In [33]:
liste[100]

['12011101171110',
 '1.2.840.113619.2.55.3.34213890.138.1392963185.715',
 '1.2.840.113619.2.55.3.34213890.138.1392963185.715_nifti_CT.nii',
 '1.2.840.113619.2.55.3.34213890.138.1392963185.715_nifti_PT.nii',
 'PET_Screening']

In [34]:
#clean serie : must have patient ID, study_UID, CT_path, PET_path, MASK_path if pet0, and type "PET" (0,2,4)
completed_liste = []
for serie in liste : 
    if len(serie) == 5 : #== 6 if Mask_path 
        completed_liste.append(serie)

In [35]:
#here check for double : if there are 2 patient with same ID, or same STUDY_UID, delete the other(s) serie(s)

dataset = []
dataset.append(completed_liste[0])
for i in range(1, len(completed_liste)) :
    if completed_liste[i] not in dataset : 
        dataset.append(completed_liste[i])

In [37]:
print("nombre de study : ", len(dataset))

nombre de study :  194


In [38]:
dataset[0]

['12011101031111',
 '1.2.250.1.74.20130606181600.17000002986493',
 '1.2.250.1.74.20130606181600.17000002986493_nifti_CT.nii',
 '1.2.250.1.74.20130606181600.17000002986493_nifti_PT.nii',
 'PET_Screening']

In [40]:
#create folder to save nifti

for serie in dataset: 
    patient_id = serie[0]
    pet_type = serie[-1]
    nifti_ct = serie[2]
    nifti_pet = serie[3]
    #nifti_mask = serie[5] #if pet0

    folder = '/media/deeplearning/Elements/RELEVANCE_NIFTI'+'/'+patient_id

    if not os.path.exists(folder):
        os.makedirs(folder)
        print(folder, 'created')
    else : print(folder, "Dossier patient déjà crée")

    subfolder = folder+'/'+pet_type
    if not os.path.exists(subfolder) :
        os.makedirs(subfolder)
        print(subfolder, 'created')
    else : print(subfolder, "Déjà creer => already folder for ", pet_type)

    ct_path = '/media/deeplearning/Elements/RELEVANCE_NIFTI'+'/'+nifti_ct
    pt_path = '/media/deeplearning/Elements/RELEVANCE_NIFTI'+'/'+nifti_pet
        #mask_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'+'/'+nifti_mask

    new_ct_path = subfolder+'/'+nifti_ct
    new_pet_path = subfolder+'/'+nifti_pet
    #new_mask_path = subfolder+'/'+nifti_mask
    try : 

        os.rename(ct_path, subfolder+'/'+nifti_ct)
        os.rename(pt_path, subfolder+'/'+nifti_pet)
        #os.rename(mask_path, subfolder+'/'+nifti_mask)


        serie[2] = new_ct_path
        serie[3] = new_pet_path 
        #serie[4] = new_mask_path
    except Exception as err : 
        print(err)


ready folder for  PET_Screening
/media/deeplearning/Elements/RELEVANCE_NIFTI/12011101071106 Dossier patient déjà crée
/media/deeplearning/Elements/RELEVANCE_NIFTI/12011101071106/PET_Screening Déjà creer => already folder for  PET_Screening
/media/deeplearning/Elements/RELEVANCE_NIFTI/12011101071107 Dossier patient déjà crée
/media/deeplearning/Elements/RELEVANCE_NIFTI/12011101071107/PET_Screening Déjà creer => already folder for  PET_Screening
/media/deeplearning/Elements/RELEVANCE_NIFTI/12011101071108 Dossier patient déjà crée
/media/deeplearning/Elements/RELEVANCE_NIFTI/12011101071108/PET_Screening Déjà creer => already folder for  PET_Screening
/media/deeplearning/Elements/RELEVANCE_NIFTI/12011101071109 Dossier patient déjà crée
/media/deeplearning/Elements/RELEVANCE_NIFTI/12011101071109/PET_Screening Déjà creer => already folder for  PET_Screening
/media/deeplearning/Elements/RELEVANCE_NIFTI/12011101071110 Dossier patient déjà crée
/media/deeplearning/Elements/RELEVANCE_NIFTI/12011

In [41]:
#write csv , add NIFTI_MASK column if there is a mask 
nifti_directory = '/media/deeplearning/Elements/RELEVANCE_NIFTI'
filename = 'RELEVANCE_NIFTI_SCREENING.csv'
with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "STUDY UID", "PET", "NIFTI_CT", "NIFTI_PET"])
    for serie in dataset: 
        csv_writer.writerow([serie[0], serie[1], serie[-1], serie[2], serie[3]])